1) test1.find_opt() - Обычный поиск
2) test1.find_opt_change_one() - Поиск с изменением границ компонентов и соотношений. Перебираются последовательно условия. Даётся 2 лучших варианта: по соотношениям, по границе компонентов. 

In [1]:
import pandas as pd
from scipy.optimize import linprog, minimize
import numpy as np

In [2]:
df=pd.read_excel('data1.xlsx')
df

,Unnamed: 0,Имя,Цена,Сухое,Натура,Сырой,Крахмал,NDF,ADF,Ca,Phos
0,Имся,единица,руб,кг,кг,кг,кг,кг,кг,кг,кг
1,Ячмень зерно,кг,10.000,0.900,1.111,0.150,0.500,0.200,0.100,0.002,0.003
2,Кукуруза зерно,кг,15.000,0.800,1.250,0.100,0.600,0.250,0.150,0.003,0.004
3,Силос,кг,5.000,0.400,2.500,0.050,0.300,0.400,0.200,0.004,0.005
4,Соя шрот,кг,20.000,0.900,1.111,0.400,0.200,0.300,0.250,0.005,0.006


In [3]:
class method1():   
    def __init__(self, products, bnd, comps, comps_bnd, matrix, costs, sootnosheniya=None):
        self.products=products
        self.bnd=bnd
        self.comps=comps
        self.comps_bnd=comps_bnd
        self.ratio=sootnosheniya
        self.main_mat=matrix[products, :][:, comps]
        cond_l=[]
        cond_r=[]
        if comps_bnd:
            for num, i in enumerate(comps_bnd):
                if i[0]:
                    cond_l.append(-self.main_mat[:, num])
                    cond_r.append(-float(i[0]))
                if i[1]:
                    cond_l.append(self.main_mat[:, num])
                    cond_r.append(float(i[1]))
        self.cond_l=np.array(cond_l)
        self.cond_r=np.array(cond_r)
        self.costs=costs
        self.result=None
        self.KOEF=1.01
        self.add=1

    def get_ratio(products, comps, sootnosheniya, main_mat):
        for_ratio=np.zeros([len(sootnosheniya), len(products)])
        for num, i in enumerate(sootnosheniya):
            chisl=0
            znam=0
            idx1=0
            idx2=0
            for num1, j in enumerate(i):
                if j!=0:
                    if j!=1 and j!=-1:
                        chisl=j
                        idx1=num1
                    else:
                        znam=j
                        idx2=num1
            for_ratio[num]=main_mat[:, idx2]*chisl-main_mat[:, idx1]*znam
        return for_ratio
        
    def find_opt(self, ratio=None, cond_r=None, mode=1):
        if ratio is None and not self.ratio is None:
            ratio=self.ratio
        if cond_r is None and len(self.cond_r)!=0:
            cond_r=self.cond_r   
        
        if not self.ratio is None and mode==1:
            if len(self.cond_l)>0:
                ratio_stats=method1.get_ratio(products, comps, ratio, self.main_mat)
                A_ub=np.concatenate([self.cond_l, ratio_stats], axis=0)
                b_ub=np.concatenate([cond_r, np.zeros(len(ratio_stats))], axis=0)
            else:
                A_ub=ratio_stats
                b_ub=self.np.zeros(len(ratio_stats))
        else:
            A_ub=self.cond_l
            b_ub=cond_r
            
        opt = linprog(c=self.costs, 
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=self.bnd,
        method="highs")
        if opt.success:
            result=np.around(np.matmul(opt.x, self.main_mat),3)
            x=np.around(opt.x, 3)
            fun=round(opt.fun, 3)           
        else:
            result=None
            x=None
            fun=None
        if not ratio is None:
            ratio=np.around(ratio, 3)       
        return {'success':opt.success, 'x': x, 'fun':fun, 'comps': result,
               'ratio': ratio, 'comps_bnd': np.around(cond_r, 3)}
               
    def find_opt_change_one(self):
        results1=[]
        for i in range(len(self.ratio)):
            ratio=np.array(self.ratio)
            cond_r=self.cond_r
            ratio[i]=np.where((ratio[i]!=0) & (ratio[i]!=1), 0.01, ratio[i])
            dis_ratio=ratio
            res=self.find_opt(ratio=dis_ratio, cond_r=cond_r)
            def funx1(ratio_val):
                ratio[i]=np.where((ratio[i]!=0) & (ratio[i]!=1), ratio_val, ratio[i])
                opt=self.find_opt(ratio=ratio, cond_r=cond_r)
                if opt['success']:
                    return np.linalg.norm(ratio-self.ratio)
                else:
                    return np.linalg.norm(self.ratio-res['ratio'])            
            if res['success']:
                for val  in res['ratio'][i]:
                    if val!=0 and val!=1:
                        break
                min_res=minimize(funx1, val, method='Nelder-Mead')
                ratio[i]=np.where((ratio[i]!=0) & (ratio[i]!=1), min_res.x, ratio[i])
                results1.append((cond_r, ratio, min_res.fun))
        results2=[]
        for i in range(len(self.cond_r)):
            ratio=np.array(self.ratio)
            cond_r=self.cond_r.copy()
            cond_r_max=cond_r.copy()
            cond_r_max[i]=np.where(cond_r[i]>0, cond_r[i]*100, 0)
            res=self.find_opt(ratio=ratio, cond_r=cond_r_max)
            def funx2(cond_i):
                cond=cond_r.copy()
                cond[i]=cond_i
                opt=self.find_opt(ratio=ratio, cond_r=cond)
                if opt['success']:
                    return np.linalg.norm(cond-self.cond_r)
                else:
                    return np.linalg.norm(self.cond_r-res['comps_bnd'])
            if res['success']:
                min_res=minimize(funx2, res['comps_bnd'][i], method='Nelder-Mead')
                cond_r[i]=min_res.x
                results2.append((cond_r, ratio, min_res.fun))
        if results1:
            idx=np.argmin([x[2] for x in results1])
            res1=self.find_opt(ratio=results1[idx][1], cond_r=results1[idx][0])
        else:
            res1=None
        if results2:
            idx=np.argmin([x[2] for x in results2])
            res2=self.find_opt(ratio=results2[idx][1], cond_r=results2[idx][0])
        else:
            res2=None
        return {'Пропорции': res1, 'Границы компонетов': res2}

In [4]:
datas=df.drop([0], axis=0).drop(['Unnamed: 0', 'Имя'], axis=1).astype(float).to_numpy()
costs=datas[:, 0]
voda=datas[:, 2:3]
componets_for_div=datas[:, 2:]
divided=componets_for_div/voda
divided=np.transpose(np.transpose(divided))
matrix=np.concatenate([datas[:,1].reshape((4, 1)), divided], axis=1)

Первый индекс описывает продукт
1	Ячмень зерно	
2	Кукуруза зерно	
3	Силос	
4	Соя шрот
Второй индекс описывает компонент (кроме натурального, ушёл как коэф.)
1 Сухое
2 Натур
3 Сырой
4 Крахмал
5 NDF
6 ADF
7 Ca
8 Phos

## Пример1

In [5]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[20, None],[1.5, None], [6, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0], [1, 0, 0.12, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12

In [6]:
test1=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)

Если условие >=, то данные идут с плюсом.
Если <=, то надо умножить c -1.

In [7]:
opt=test1.find_opt()
opt

{'success': False,
 'x': None,
 'fun': None,
 'comps': None,
 'ratio': array([[0.6 , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.12, 0.  ]]),
 'comps_bnd': array([-10. , -20. ,  -1.5,  -6. ])}

In [8]:
res1=test1.find_opt_change_one()
res1

{'Пропорции': {'success': True,
  'x': array([ 5., 10.,  5.]),
  'fun': 224.997,
  'comps': array([14.5  , 20.   ,  1.575,  7.65 ]),
  'ratio': array([[0.6  , 1.   , 0.   , 0.   ],
         [1.   , 0.   , 0.109, 0.   ]]),
  'comps_bnd': array([-10. , -20. ,  -1.5,  -6. ])},
 'Границы компонетов': None}

## Пример 2

In [9]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[None, None],[2, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12
test2=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)

In [10]:
opt=test2.find_opt()
opt

{'success': False,
 'x': None,
 'fun': None,
 'comps': None,
 'ratio': array([[0.6, 1. , 0. , 0. ]]),
 'comps_bnd': array([-10.,  -2.])}

In [11]:
opt=test2.find_opt_change_one()
opt

{'Пропорции': {'success': True,
  'x': array([ 5.   , 10.   , 26.247]),
  'fun': 331.233,
  'comps': array([22.999, 41.247,  2.   , 10.2  ]),
  'ratio': array([[0.558, 1.   , 0.   , 0.   ]]),
  'comps_bnd': array([-10.,  -2.])},
 'Границы компонетов': {'success': True,
  'x': array([ 5.   , 10.   , 17.497]),
  'fun': 287.483,
  'comps': array([19.499, 32.497,  1.825,  9.15 ]),
  'ratio': array([[0.6, 1. , 0. , 0. ]]),
  'comps_bnd': array([-10.   ,  -1.825])}}

## Пример 3

In [12]:
products=[0, 1, 2]#Номера продуктов
bnd=[[5, 5], [8.75, 8.75], [1.25, 1.25]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[15, None],[1.3, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.8, 1, 0, 0],  [0.2, 0, 0.0, 1]]#соотношения компонентов
test3=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)

In [13]:
opt=test3.find_opt()
opt

{'success': True,
 'x': array([5.  , 8.75, 1.25]),
 'fun': 187.5,
 'comps': array([12. , 15. ,  1.4,  6.6]),
 'ratio': array([[0.8, 1. , 0. , 0. ],
        [0.2, 0. , 0. , 1. ]]),
 'comps_bnd': array([-10. , -15. ,  -1.3])}

In [14]:
opt=test3.find_opt_change_one()
opt

{'Пропорции': {'success': True,
  'x': array([5.  , 8.75, 1.25]),
  'fun': 187.5,
  'comps': array([12. , 15. ,  1.4,  6.6]),
  'ratio': array([[0.8, 1. , 0. , 0. ],
         [0.2, 0. , 0. , 1. ]]),
  'comps_bnd': array([-10. , -15. ,  -1.3])},
 'Границы компонетов': {'success': True,
  'x': array([5.  , 8.75, 1.25]),
  'fun': 187.5,
  'comps': array([12. , 15. ,  1.4,  6.6]),
  'ratio': array([[0.8, 1. , 0. , 0. ],
         [0.2, 0. , 0. , 1. ]]),
  'comps_bnd': array([-10. , -15. ,  -1.3])}}